In [ ]:
import sys, os, time, glob
import numpy as np
from datetime import datetime
from statistics import median
import matplotlib.pyplot as plt
# import statsmodels.api as sm
import csv
from csv import writer
from csv import reader
import pandas as pd 

from sklearn.model_selection import GridSearchCV
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import norm
from sklearn.neighbors import KernelDensity

In [ ]:

dir_name = '/lcrc/project/EE-ECP/jkoo/code/sdv/Benchmarks/'
kernel_dir = '3mm_void'
kernel = '3mm'
# kernel = 'covariance'
# kernel_dir = 'covariance_void'
# kernel_dir = 'heat-3d_void'
# kernel = 'heat-3d'
# kernel_dir = 'floyd-warshall_void'
# kernel = 'floyd-warshall'
# kernel_dir = 'lu_void'
# kernel = 'lu'
# kernel_dir = 'syr2k_void'
# kernel = 'syr2k'



d_sizes = ['s','sm','m','ml','l','xl']
o3p = {}
for s in d_sizes: 
    time = pd.read_csv(f'{dir_name}{kernel}_exp/{kernel_dir}/results_{s}_{kernel}_void.csv')['objective']
    o3p[s] = float(time)

o3p

In [ ]:
dir_name = '/lcrc/project/EE-ECP/jkoo/code/ytopt/ytopt/benchmark/polybench-exp'
dir_name_gptune = '/lcrc/project/EE-ECP/jkoo/code/gptune/examples'

d_sizes = ['s','m','l','sm','ml','xl']
gathered = []
cutoffs = []

gathered = [] 
cutoffs = [] 
tag1 = ''#_1.96_gp'# '_1.96_lcb'# 
tag2 = '' # '_19.6_lcb'#_1.96_gp_n30'# '_11.0_lcb' # 
tag3 = ''
tag4 = ''#_1.96_gp'# '_1.96_lcb'# 
tag5 = ''#_1.96_gp_n30'# '_11.0_lcb' # 

for d_size, o3 in zip(d_sizes, o3p):   
    
    alpha = 0.1
    t = 0
    evals = []    
    
    ### BO
    file_n = f'{dir_name}/{kernel}/results_rf_{d_size}_{kernel}{tag1}.csv'
    evals_bo = o3p[d_size] / np.array(pd.read_csv(file_n)['objective']) 

    ## RS 
    file_n = f'{dir_name}/{kernel}/results_rs_{d_size}_{kernel}{tag1}.csv'
    evals_rs = o3p[d_size] / np.array(pd.read_csv(file_n)['objective']) 
    
    
    # Plot a 1D density example
    N = len(evals_rs)

    # X_0  = times_0.values[:, np.newaxis]
    X_rf = evals_bo[:, np.newaxis]#.values[:, np.newaxis]
    X    = evals_rs[:, np.newaxis]#.values[:, np.newaxis]

    # print (X.shape)
    X_plot = np.linspace(min(X.min(),X_rf.min())*0.999, max(X.max(),X_rf.max())*1.001, 1000)[:, np.newaxis]

    fig, ax = plt.subplots()
    # ax.fill(X_plot[:, 0], true_dens, fc="black", alpha=0.2, label="input distribution")
    colors = ["navy"]#, "darkorange"]
    colors = ["navy", "darkorange", "green"]
    kernels = ["gaussian"]#,"gaussian"]#, "tophat"]#, "epanechnikov"]
    lw = 2

    # for x, color, s in zip([X],colors, ['RS']):
    bandwidth_sizes = []
    for x, color, s in zip([X, X_rf],colors, ['RS','RF']):
    # for x, color, s in zip([X_0, X, X_rf],colors, ['RS','RS_p0_fixed_rerun','RS_p0p3_fixed']):
    # for x, color, s in zip([X],colors, ['RS']):
        for k in kernels:
    #         find best bandwidth
            grid = GridSearchCV(KernelDensity(),
                        {'bandwidth': np.linspace(0.001, 10, 100)},
                        cv=max(2,round(len(x)*0.2))) # 20-fold cross-validation
            grid.fit(x)
#             print ('grid.best_params_',grid.best_params_)
            bandwidth_size = grid.best_params_['bandwidth']
            bandwidth_sizes.append(bandwidth_size)
#             bandwidth_size = 0.1

    bandwidth_size = np.mean(bandwidth_sizes)
    print ('chosen',bandwidth_size)
    for x, color, s in zip([X, X_rf],colors, ['RS','RF']):
     
        for k in kernels:
            kde = KernelDensity(kernel=k, bandwidth=bandwidth_size).fit(x)
            log_dens = kde.score_samples(X_plot)
            ax.plot(
                X_plot[:, 0],
                np.exp(log_dens),
                color=color,
                lw=lw,
                linestyle="-",
                label="Search = '{0}'".format(s),
            )

        # ax.text(6, 0.38, "N={0} points".format(N))

        ax.legend(loc="upper left")
        ax.plot(x[:, 0], -0.005 - 0.01 * np.random.random(x.shape[0]), "+k", color=color)



    # ax.set_xlim(-4, 9)
    # ax.set_ylim(-0.02, 1.0)
    # plt.title('sw4lite CPU')
    # plt.title('XSBench CPU',fontsize=14)
    # plt.title('RSBench CPU',fontsize=14)
    plt.title(kernel+'_'+d_size,fontsize=14)
    plt.xlabel('Time')
    plt.xlabel('Speedup over O3',fontsize=14)
    plt.ylabel('Density',fontsize=14)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.legend(fontsize=12)
    plt.show()    
    

In [ ]:
# o3p = [0.3243315, 3.258446, 16.2930585] # xsbench cpu

# o3p = [6.5066349,12.19831,17.96101]     ## xsbench gpu s m l  

# o3p = [1.916425, 19.4219, ,97.223435 ] # rsbench cpu
# o3p = [48.421005] ## rsbench gpu s m l  

### sw4lite cpu
## time s1 32.9947499999999 ###   
## time s3 78.30985

### sw4lite gpu
## time s9: 10.7415
## time s3: 6.30483

O3 = 97.223435 # 97.223435

#################### polybencch cpu large 
### 3mm 0.4220245
###

O3 = {}
O3['3mm'] = 0.4220245
O3['covariance'] = 2.157349
O3['heat-3d'] =2.801542
O3['lu'] = 7.604848
O3['syr2k'] = 0.4827285

o3 = O3[Kernel]

# times_0    = O3 / df_0['objective']
times    = o3 / df['objective']
times_rf = o3 / df_rf['objective']

# times    =  df['objective']
# times_rf =  df_rf['objective']


# times = 10.74 / df[df['objective'] <= 100]['objective'] 
# times_rf = df_rf['objective']

In [ ]:
from sklearn.model_selection import GridSearchCV
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import norm
from sklearn.neighbors import KernelDensity
# ----------------------------------------------------------------------
# Plot a 1D density example
N = 200

# X_0  = times_0.values[:, np.newaxis]
X    = times.values[:, np.newaxis]
X_rf = times_rf.values[:, np.newaxis]

# print (X.shape)
X_plot = np.linspace(X.min()*0.999, X_rf.max()*1.001, 1000)[:, np.newaxis]

fig, ax = plt.subplots()
# ax.fill(X_plot[:, 0], true_dens, fc="black", alpha=0.2, label="input distribution")
colors = ["navy"]#, "darkorange"]
colors = ["navy", "darkorange", "green"]
kernels = ["gaussian"]#,"gaussian"]#, "tophat"]#, "epanechnikov"]
lw = 2

# for x, color, s in zip([X],colors, ['RS']):
for x, color, s in zip([X, X_rf],colors, ['RS','RF']):
# for x, color, s in zip([X_0, X, X_rf],colors, ['RS','RS_p0_fixed_rerun','RS_p0p3_fixed']):
# for x, color, s in zip([X],colors, ['RS']):
    for kernel in kernels:
#         find best bandwidth
        grid = GridSearchCV(KernelDensity(),
                    {'bandwidth': np.linspace(0.001, 10, 1000)},
                    cv=max(2,round(len(x)*0.2))) # 20-fold cross-validation
        grid.fit(x)
        print ('grid.best_params_',grid.best_params_)
        bandwidth_size = grid.best_params_['bandwidth']
#         bandwidth_size = 0.03
        kde = KernelDensity(kernel=kernel, bandwidth=bandwidth_size).fit(x)
        log_dens = kde.score_samples(X_plot)
        ax.plot(
            X_plot[:, 0],
            np.exp(log_dens),
            color=color,
            lw=lw,
            linestyle="-",
            label="Search = '{0}'".format(s),
        )

    # ax.text(6, 0.38, "N={0} points".format(N))

    ax.legend(loc="upper left")
    ax.plot(x[:, 0], -0.005 - 0.01 * np.random.random(x.shape[0]), "+k", color=color)



# ax.set_xlim(-4, 9)
# ax.set_ylim(-0.02, 1.0)
# plt.title('sw4lite CPU')
# plt.title('XSBench CPU',fontsize=14)
# plt.title('RSBench CPU',fontsize=14)
plt.title(Kernel+' CPU',fontsize=14)
plt.xlabel('Time')
plt.xlabel('Speedup over O3',fontsize=14)
plt.ylabel('Density',fontsize=14)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(fontsize=12)
plt.show()

In [ ]:
fig = plt.figure()
fig, ax = plt.subplots()
# ax.plot(df.timestamp_end, df.Run_time)
ax.plot(df.elapsed_sec, df.objective,'.-' ,label='RS')
ax.plot(df_rf.elapsed_sec, df_rf.objective,'.-' ,label='RF')
# ax.plot(df.elapsed_sec, df.objective,'.-' )
ax.set_ylabel('Execution time',fontsize=14)
ax.set_xlabel('Time (s.)',fontsize=14)
ax.set_xlim(0)
# ax.set_ylim(0)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(fontsize=12)
# plt.title('XSBench CPU',fontsize=14)
plt.title(Kernel+' CPU',fontsize=14)
# plt.title('sw4lite CPU',fontsize=14)
ax.grid()
fig.show()

In [ ]:
df.corr()

In [ ]:
# ----------------------------------------------------------------------
# Plot the progression of histograms to kernels
np.random.seed(1)
# N = 20
# X = np.concatenate(
#     (np.random.normal(0, 1, int(0.3 * N)), np.random.normal(5, 1, int(0.7 * N)))
# )[:, np.newaxis]
# X_plot = np.linspace(-5, 10, 1000)[:, np.newaxis]
# bins = np.linspace(-5, 10, 10)
density_param = {"density": True}
N = 200
np.random.seed(1)
X = np.concatenate(
    (np.random.normal(0, 1, int(0.3 * N)), np.random.normal(5, 1, int(0.7 * N)))
)[:, np.newaxis]
X = times.values[:, np.newaxis]
X_plot = np.linspace(X.min()*0.0, X.max()*1.1, 1000)[:, np.newaxis]
bins = np.linspace(X.min()*0.0, X.max()*1.1, 100)

fig, ax = plt.subplots(2, 2, sharex=True, sharey=True)
fig.subplots_adjust(hspace=0.05, wspace=0.05)

# histogram 1
ax[0, 0].hist(X[:, 0], bins=bins, fc="#AAAAFF", **density_param)
ax[0, 0].text(-3.5, 0.31, "Histogram")

# histogram 2
ax[0, 1].hist(X[:, 0], bins=bins + 0.75, fc="#AAAAFF", **density_param)
ax[0, 1].text(-3.5, 0.31, "Histogram, bins shifted")

# tophat KDE
kde = KernelDensity(kernel="tophat", bandwidth=0.75).fit(X)
log_dens = kde.score_samples(X_plot)
ax[1, 0].fill(X_plot[:, 0], np.exp(log_dens), fc="#AAAAFF")
ax[1, 0].text(-3.5, 0.31, "Tophat Kernel Density")

# Gaussian KDE
kde = KernelDensity(kernel="gaussian", bandwidth=0.75).fit(X)
log_dens = kde.score_samples(X_plot)
ax[1, 1].fill(X_plot[:, 0], np.exp(log_dens), fc="#AAAAFF")
ax[1, 1].text(-3.5, 0.31, "Gaussian Kernel Density")

for axi in ax.ravel():
    axi.plot(X[:, 0], np.full(X.shape[0], -0.01), "+k")
#     axi.set_xlim(-4, 9)
#     axi.set_ylim(-0.02, 0.34)

for axi in ax[:, 0]:
    axi.set_ylabel("Normalized Density")

for axi in ax[1, :]:
    axi.set_xlabel("x")

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt